In [1]:
%tensorflow_version 2.x

In [3]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-12-31 06:52:57--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-12-31 06:52:57--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2391c4c86194fd303b49f487fe.dl.dropboxusercontent.com/zip_download_get/ApfNvl7QqK2XOUbYfUdrVq_sRPk8JqN1hr91fbLVUkZh1EKpZHtUA2id19xdmLC0yw1OFr9Fnc-877TH7rEZ1rl3J180DY6eD945IFZVN2leCA [following]
--2020-12-31 06:52:57--  https://uc2391c4c86194fd303b49f487fe.dl.dropboxusercontent.com/zip_download_get/ApfNvl7QqK2XOUbYfUdrVq_sRPk8JqN1hr91fbLVUkZh1EKpZHtUA2id19xdmLC0yw1OFr9Fnc-877

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# file = '../input/spam-data/train.csv'
file ='./train.csv'
dataset = pd.read_csv(file)

In [10]:
dataset.count()

qid              1306122
question_text    1306122
target           1306122
dtype: int64

In [ ]:
# def to_lower(word):
#     result = word.lower()
#     return result

# def remove_special_characters(word):
#     result= word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
#     return result

# def remove_stop_words(words):
#     result = [i for i in words if i not in ENGLISH_STOP_WORDS]
#     return result

# def remove_hyperlink(word):
#     return re.sub(r"http\S+", "", word)


In [11]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

texts = []
for i in range(0,10000):
    mail = re.sub('[^a-zA-Z]',' ',dataset['question_text'][i])
    mail = mail.lower()
    mail = mail.split()
    ps = PorterStemmer()
    mail = [ps.stem(word) for word in mail 
            if not word in set(stopwords.words('english','french'))]
    mail = ' '.join(mail)
    texts.append(mail)
    print(texts)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['quebec nationalist see provinc nation']
['quebec nationalist see provinc nation', 'adopt dog would encourag peopl adopt shop']
['quebec nationalist see provinc nation', 'adopt dog would encourag peopl adopt shop', 'veloc affect time veloc affect space geometri']
['quebec nationalist see provinc nation', 'adopt dog would encourag peopl adopt shop', 'veloc affect time veloc affect space geometri', 'otto von guerick use magdeburg hemispher']
['quebec nationalist see provinc nation', 'adopt dog would encourag peopl adopt shop', 'veloc affect time veloc affect space geometri', 'otto von guerick use magdeburg hemispher', 'convert montra helicon mountain bike chang tyre']
['quebec nationalist see provinc nation', 'adopt dog would encourag peopl adopt shop', 'veloc affect time veloc affect space geometri', 'otto von guerick use magdeburg hemispher', 'convert montra helicon mountain 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
texts[1:10]

['adopt dog would encourag peopl adopt shop',
 'veloc affect time veloc affect space geometri',
 'otto von guerick use magdeburg hemispher',
 'convert montra helicon mountain bike chang tyre',
 'gaza slowli becom auschwitz dachau treblinka palestinian',
 'quora automat ban conserv opinion report liber view',
 'crazi wash wipe groceri germ everywher',
 'thing dress moder differ dress modestli',
 'ever phase wherein becam ignor peopl love complet disregard feel live get someth go way feel temporarili eas thing chang']

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(texts).toarray()
y = dataset.iloc[:10000,-1].values

In [14]:
y[1:1000]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [16]:
from keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(x_train,maxlen=40)
x_test = pad_sequences(x_test,maxlen=40)

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# y_test = le.transform(y_test)

In [17]:
import tensorflow as tf
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

#size of the output vector from each layer
embedding_vector_length = 32

 

In [18]:
max_feature = 10000
#Creating a sequential model
model = tf.keras.Sequential()

#Creating an embedding layer to vectorize
model.add(Embedding(max_feature, embedding_vector_length, input_length=40))

#Addding Bi-directional LSTM
model.add(Bidirectional(tf.keras.layers.LSTM(64)))

In [19]:
#Relu allows converging quickly and allows backpropagation
model.add(Dense(16, activation='relu'))

#Deep Learninng models can be overfit easily, to avoid this, we add randomization using drop out
model.add(Dropout(0.1))

#Adding sigmoid activation function to normalize the output
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 32)            320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               49664     
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 371,745
Trainable params: 371,745
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
history = model.fit(x_train, y_train, batch_size=512, epochs=20, validation_data=(x_test, y_test))

y_predict = [1 if o>0.5 else 0 for o in model.predict(x_test)]


Epoch 1/20
16/16 [==============================] - 8s 312ms/step - loss: 0.6588 - accuracy: 0.7561 - val_loss: 0.3521 - val_accuracy: 0.9380
Epoch 2/20
16/16 [==============================] - 4s 255ms/step - loss: 0.2937 - accuracy: 0.9353 - val_loss: 0.2324 - val_accuracy: 0.9380
Epoch 3/20
16/16 [==============================] - 4s 243ms/step - loss: 0.2635 - accuracy: 0.9330 - val_loss: 0.2340 - val_accuracy: 0.9380
Epoch 4/20
16/16 [==============================] - 4s 256ms/step - loss: 0.2561 - accuracy: 0.9332 - val_loss: 0.2326 - val_accuracy: 0.9380
Epoch 5/20
16/16 [==============================] - 4s 243ms/step - loss: 0.2539 - accuracy: 0.9336 - val_loss: 0.2324 - val_accuracy: 0.9380
Epoch 6/20
16/16 [==============================] - 4s 248ms/step - loss: 0.2478 - accuracy: 0.9362 - val_loss: 0.2325 - val_accuracy: 0.9380
Epoch 7/20
16/16 [==============================] - 4s 246ms/step - loss: 0.2520 - accuracy: 0.9336 - val_loss: 0.2324 - val_accuracy: 0.9380
Epoch 

In [21]:
from sklearn.metrics import confusion_matrix,accuracy_score

cf_matrix =confusion_matrix(y_test,y_predict)

print(cf_matrix)

score =accuracy_score(y_test,y_predict)

print(score)


[[1876    0]
 [ 124    0]]
0.938
